In [1]:
## -------------------------------------------------------------- ##
## Usar esta configuración                                        ##
## -------------------------------------------------------------- ##
import pyspark
import os
os.environ["PYSPARK_PYTHON"] = "python3.6"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python3.6"
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("sample_app")
sc = SparkContext(conf=conf)

In [ ]:
integersListRDD = sc.parallelize([1,2,3,4,5,6,7,8,9,10], 8)

In [ ]:
type(integersListRDD)

In [4]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

path_to_datos_navent = '/Users/ignacio.iglesias/Dev/datos/finger-data/datos_navent_fiuba/'

df_vistas = sqlContext.read.csv(path_to_datos_navent + 'fiuba_3_vistas.csv', header=True)

In [ ]:
df_vistas

## 1) Dado el siguiente código de pyspark e ignorando la definicion de pairRDD 

`print pairRDD.groupByKey().mapValues(lambda x: sum(x)).collect() print pairRDD.reduceByKey(add).collect()` 

Obtenemos la siguiente salida

`[('a', 3), ('b', 1)] [('a', 3), ('b', 1)]`

¿Como debería estar definida pairRDD para obtener ese resultado?

In [ ]:
#pairRDD =  [('a', 1), ('a', 1), ('a', 1), ('b', 1)]
#pairRDD = sc.parallelize([('a', 1), ('a', 1), ('a', 1), ('b', 0.5)])
#pairRDD = sc.parallelize([('b', 1), ('b', 1), ('b', 1), ('a', 2)])
pairRDD = sc.parallelize([('a', 1), ('a', 2), ('b', 1)])
print(pairRDD.groupByKey().mapValues(lambda x: sum(x)).collect())
print(pairRDD.reduceByKey(add).collect())

In [ ]:
def pepe(x, y):
    return x + y

pairRDD.reduceByKey(pepe).collect()

## 3) Tengo un RDD con los siguientes campos: 

`padron, apellido_nombre, fecha_nacimiento, fecha_ingreso, codigo_carrera.`

In [17]:
rdd = sqlContext.read.csv('test.csv', header=True)
type(rdd)
rdd = rdd.rdd

In [19]:
#rdd.filter(rdd['codigo_carrera'] == 10)\
#   .filter(rdd['fecha_ingreso'] > '2016-01-01')\
#    .map(lambda row: (len(row.apellido_nombre), 1))\
#     .reduceByKey(lambda a, b: a+b).collect()
         
rdd.filter(lambda row: row.codigo_carrera == '10')\
   .filter(lambda row: row.fecha_ingreso > '2016-01-01')\
    .map(lambda row: (len(row.apellido_nombre), 1))\
    .reduceByKey(lambda a, b: a+b).collect()            
            
# obtiene los apellidos de los alumnos que hayan ingresado despues del 2016-01-01 a la carrera 10
        

[(12, 1)]

In [25]:
from datetime import date

rdd.filter(lambda row: row.fecha_nacimiento > '1970-01-01')\
   .filter(lambda row: row.fecha_ingreso > '2010-01-01')\
    .map(lambda row: (date.today().year - int(row.fecha_nacimiento[:4]), 1))\
    .reduce(lambda a, b: (a[0] + b[0], a[1] + b[1]))
    
#  obtiene la cantidad de alumnos con fecha de nacimiento anterior al 1970 
# y que hayan ingresado despues del 1 de Enero de 2010.    

(2, 2)

In [26]:
rdd.filter(lambda row: row.fecha_nacimiento > '1970-01-01')\
.filter(lambda row: row.fecha_ingreso > '2010-01-01')\
.map(lambda row: (date.today().year - int(row.fecha_nacimiento[:4]), 1))\
.reduce(lambda a, b: (a[0] + b[0], a[1] + b[1])) 

# obtiene la edad de los alumnos con fecha de nacimiento anterior al 1970
# y que hayan ingresado despues del 1 de Enero de 2010

(2, 2)

In [27]:
rdd.map(lambda row: (row.fecha_ingreso[:4], 1))\
.reduceByKey(lambda a, b: a+b).collect() 

# obtiene la cantidad de inscriptos por año.

[('2017', 2)]

In [22]:
rdd.take(2)

[Row(padron='1111', apellido_nombre='GARCIA Pablo', fecha_nacimiento='2017-01-01', fecha_ingreso='2017-01-01', codigo_carrera='10'),
 Row(padron='2222', apellido_nombre='PEREZ Gaston', fecha_nacimiento='2017-01-01', fecha_ingreso='2017-01-01', codigo_carrera='20')]

In [ ]:
rdd.filter(lambda row: row.fecha_nacimiento < '1970-01-01')\
   .filter(lambda row: row.fecha_ingreso > '2010-01-01')\
    .map(lambda row: (date.today().year - row.fecha_nacimiento.year, 1))\
    .reduce(lambda a, b: (a[0] + b[0], a[1] + b[1])) 
    
# obtiene la cantidad de alumnos con fecha de nacimiento anterior al 1970
# y que hayan ingresado despues del 1 de Enero de 2010

In [ ]:
'2017-01-01' < '1970-01-01'

In [ ]:
from datetime import date

def lowerThan1970(row):
    return row.fecha_nacimiento < '1970-01-01'

def graterThan2010(row):
    return row.fecha_ingreso > '2010-01-01'

def yearDiff(row):
    return (date.today().year - row.fecha_nacimiento.year, 1)

def doSum(a, b):
    return (a[0] + b[0], a[1] + b[1])

rdd.filter(lowerThan1970).collect()#\
#.filter(graterThan2010)\
#.map(yearDiff)\
#.reduce(doSum) 

# obtiene la edad de los alumnos con fecha de nacimiento anterior al 1970 
# y que hayan ingresado despues del 1 de Enero de 2010.

## 4) Archivo.txt:

`datos spark guia
spark hadoop 
guia datos spark gradience`

Corremos:

`sc.textFile("Archivo.txt")
.flatMap(lambda a: a.split())
.map(lambda a: (a, 1))
.reduceByKey(lambda a, b: a+b)
.takeOrdered(3, key = lambda x: -x[1])`

In [2]:
sc.textFile("Archivo.txt")\
.flatMap(lambda a: a.split())\
.map(lambda a: (a, 1))\
.reduceByKey(lambda a, b: a + b)\
.takeOrdered(3, key = lambda x: -x[1])\

[('spark', 3), ('datos', 2), ('guia', 2)]